10 Minutes to cuDF and Dask-cuDF
=======================

Modeled after 10 Minutes to Pandas, this is a short introduction to cuDF and Dask-cuDF, geared mainly for new users.

### What are these Libraries?

[cuDF](https://github.com/rapidsai/cudf) is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API.

[Dask](https://dask.org/) is a flexible library for parallel computing in Python that makes scaling out your workflow smooth and simple. On the CPU, Dask uses Pandas to execute operations in parallel on DataFrame partitions.

[Dask-cuDF](https://github.com/rapidsai/cudf/tree/main/python/dask_cudf) extends Dask where necessary to allow its DataFrame partitions to be processed by cuDF GPU DataFrames as opposed to Pandas DataFrames. For instance, when you call dask_cudf.read_csv(...), your cluster’s GPUs do the work of parsing the CSV file(s) with underlying cudf.read_csv().


### When to use cuDF and Dask-cuDF

If your workflow is fast enough on a single GPU or your data comfortably fits in memory on a single GPU, you would want to use cuDF. If you want to distribute your workflow across multiple GPUs, have more data than you can fit in memory on a single GPU, or want to analyze data spread across many files at once, you would want to use Dask-cuDF.

In [1]:
import os

import cupy as cp
import pandas as pd
import cudf
import dask_cudf

cp.random.seed(12)

#### Portions of this were borrowed and adapted from the
#### cuDF cheatsheet, existing cuDF documentation,
#### and 10 Minutes to Pandas.

Object Creation
---------------

Creating a `cudf.Series` and `dask_cudf.Series`.

In [2]:
s = cudf.Series([1,2,3,None,4])
s

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64

In [3]:
ds = dask_cudf.from_cudf(s, npartitions=2) 
ds.compute()

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64

Creating a `cudf.DataFrame` and a `dask_cudf.DataFrame` by specifying values for each column.

In [4]:
df = cudf.DataFrame({'a': list(range(20)),
                     'b': list(reversed(range(20))),
                     'c': list(range(20))
                    })
df

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3
4,4,15,4
5,5,14,5
6,6,13,6
7,7,12,7
8,8,11,8
9,9,10,9


In [5]:
ddf = dask_cudf.from_cudf(df, npartitions=2) 
ddf.compute()

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3
4,4,15,4
5,5,14,5
6,6,13,6
7,7,12,7
8,8,11,8
9,9,10,9


Creating a `cudf.DataFrame` from a pandas `Dataframe` and a `dask_cudf.Dataframe` from a `cudf.Dataframe`.

*Note that best practice for using Dask-cuDF is to read data directly into a `dask_cudf.DataFrame` with something like `read_csv` (discussed below).*

In [6]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
gdf

,a,b
0,0,0.1
1,1,0.2
2,2,<NA>
3,3,0.3


In [7]:
dask_gdf = dask_cudf.from_cudf(gdf, npartitions=2)
dask_gdf.compute()

,a,b
0,0,0.1
1,1,0.2
2,2,<NA>
3,3,0.3


Viewing Data
-------------

Viewing the top rows of a GPU dataframe.

In [8]:
df.head(2)

,a,b,c
0,0,19,0
1,1,18,1


In [9]:
ddf.head(2)

,a,b,c
0,0,19,0
1,1,18,1


Sorting by values.

In [10]:
df.sort_values(by='b')

,a,b,c
19,19,0,19
18,18,1,18
17,17,2,17
16,16,3,16
15,15,4,15
14,14,5,14
13,13,6,13
12,12,7,12
11,11,8,11
10,10,9,10


In [11]:
ddf.sort_values(by='b').compute()

,a,b,c
19,19,0,19
18,18,1,18
17,17,2,17
16,16,3,16
15,15,4,15
14,14,5,14
13,13,6,13
12,12,7,12
11,11,8,11
10,10,9,10


Selection
------------

## Getting

Selecting a single column, which initially yields a `cudf.Series` or `dask_cudf.Series`. Calling `compute` results in a `cudf.Series` (equivalent to `df.a`).

In [12]:
df['a']

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: a, dtype: int64

In [13]:
ddf['a'].compute()

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: a, dtype: int64

## Selection by Label

Selecting rows from index 2 to index 5 from columns 'a' and 'b'.

In [14]:
df.loc[2:5, ['a', 'b']]

,a,b
2,2,17
3,3,16
4,4,15
5,5,14


In [15]:
ddf.loc[2:5, ['a', 'b']].compute()

,a,b
2,2,17
3,3,16
4,4,15
5,5,14


## Selection by Position

Selecting via integers and integer slices, like numpy/pandas. Note that this functionality is not available for Dask-cuDF DataFrames.

In [16]:
df.iloc[0]

a     0
b    19
c     0
Name: 0, dtype: int64

In [17]:
df.iloc[0:3, 0:2]

,a,b
0,0,19
1,1,18
2,2,17


You can also select elements of a `DataFrame` or `Series` with direct index access.

In [18]:
df[3:5]

,a,b,c
3,3,16,3
4,4,15,4


In [19]:
s[3:5]

3    <NA>
4       4
dtype: int64

## Boolean Indexing

Selecting rows in a `DataFrame` or `Series` by direct Boolean indexing.

In [20]:
df[df.b > 15]

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3


In [21]:
ddf[ddf.b > 15].compute()

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3


Selecting values from a `DataFrame` where a Boolean condition is met, via the `query` API.

In [22]:
df.query("b == 3")

,a,b,c
16,16,3,16


In [23]:
ddf.query("b == 3").compute()

,a,b,c
16,16,3,16


You can also pass local variables to Dask-cuDF queries, via the `local_dict` keyword. With standard cuDF, you may either use the `local_dict` keyword or directly pass the variable via the `@` keyword. Supported logical operators include `>`, `<`, `>=`, `<=`, `==`, and `!=`.

In [24]:
cudf_comparator = 3
df.query("b == @cudf_comparator")

,a,b,c
16,16,3,16


In [25]:
dask_cudf_comparator = 3
ddf.query("b == @val", local_dict={'val':dask_cudf_comparator}).compute()

,a,b,c
16,16,3,16


Using the `isin` method for filtering.

In [26]:
df[df.a.isin([0, 5])]

,a,b,c
0,0,19,0
5,5,14,5


## MultiIndex

cuDF supports hierarchical indexing of DataFrames using MultiIndex. Grouping hierarchically (see `Grouping` below) automatically produces a DataFrame with a MultiIndex.

In [27]:
arrays = [['a', 'a', 'b', 'b'], [1, 2, 3, 4]]
tuples = list(zip(*arrays))
idx = cudf.MultiIndex.from_tuples(tuples)
idx

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 3),
            ('b', 4)],
           )

This index can back either axis of a DataFrame.

In [28]:
gdf1 = cudf.DataFrame({'first': cp.random.rand(4), 'second': cp.random.rand(4)})
gdf1.index = idx
gdf1

first    second
a 1  0.082654  0.967955
  2  0.399417  0.441425
b 3  0.784297  0.793582
  4  0.070303  0.271711

In [29]:
gdf2 = cudf.DataFrame({'first': cp.random.rand(4), 'second': cp.random.rand(4)}).T
gdf2.columns = idx
gdf2

a                  b          
               1         2        3         4
first   0.343382  0.003700  0.20043  0.581614
second  0.907812  0.101512  0.24179  0.224180

Accessing values of a DataFrame with a MultiIndex. Note that slicing is not yet supported.

In [30]:
gdf1.loc[('b', 3)]

first     0.784297
second    0.793582
Name: ('b', 3), dtype: float64

Missing Data
------------

Missing data can be replaced by using the `fillna` method.

In [31]:
s.fillna(999)

0      1
1      2
2      3
3    999
4      4
dtype: int64

In [32]:
ds.fillna(999).compute()

0      1
1      2
2      3
3    999
4      4
dtype: int64

Operations
------------

## Stats

Calculating descriptive statistics for a `Series`.

In [33]:
s.mean(), s.var()

(2.5, 1.666666666666666)

In [34]:
ds.mean().compute(), ds.var().compute()

(2.5, 1.6666666666666667)

## Applymap

Applying functions to a `Series`. Note that applying user defined functions directly with Dask-cuDF is not yet implemented. For now, you can use [map_partitions](http://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.map_partitions.html) to apply a function to each partition of the distributed dataframe.

In [35]:
def add_ten(num):
    return num + 10

df['a'].applymap(add_ten)

/home/mmccarty/miniconda3/envs/cudf_dev/lib/python3.8/site-packages/cudf/core/series.py:2223: FutureWarning: Series.applymap is deprecated and will be removed in a future cuDF release. Use Series.apply instead.
  warnings.warn(


0     10
1     11
2     12
3     13
4     14
5     15
6     16
7     17
8     18
9     19
10    20
11    21
12    22
13    23
14    24
15    25
16    26
17    27
18    28
19    29
Name: a, dtype: int64

In [36]:
ddf['a'].map_partitions(add_ten).compute()

0     10
1     11
2     12
3     13
4     14
5     15
6     16
7     17
8     18
9     19
10    20
11    21
12    22
13    23
14    24
15    25
16    26
17    27
18    28
19    29
Name: a, dtype: int64

## Histogramming

Counting the number of occurrences of each unique value of variable.

In [37]:
df.a.value_counts()

15    1
6     1
1     1
14    1
2     1
5     1
11    1
7     1
17    1
13    1
8     1
16    1
0     1
10    1
4     1
9     1
19    1
18    1
3     1
12    1
Name: a, dtype: int32

In [38]:
ddf.a.value_counts().compute()

15    1
6     1
1     1
14    1
2     1
5     1
11    1
7     1
17    1
13    1
8     1
16    1
0     1
10    1
4     1
9     1
19    1
18    1
3     1
12    1
Name: a, dtype: int64

## String Methods

Like pandas, cuDF provides string processing methods in the `str` attribute of `Series`. Full documentation of string methods is a work in progress. Please see the cuDF API documentation for more information.

In [39]:
s = cudf.Series(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'])
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: object

In [40]:
ds = dask_cudf.from_cudf(s, npartitions=2)
ds.str.lower().compute()

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: object

## Concat

Concatenating `Series` and `DataFrames` row-wise.

In [41]:
s = cudf.Series([1, 2, 3, None, 5])
cudf.concat([s, s])

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64

In [42]:
ds2 = dask_cudf.from_cudf(s, npartitions=2)
dask_cudf.concat([ds2, ds2]).compute()

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64

## Join

Performing SQL style merges. Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [43]:
df_a = cudf.DataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = cudf.DataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left')
merged

,key,vals_a,vals_b
0,a,10.0,100.0
1,c,12.0,101.0
2,e,14.0,102.0
3,b,11.0,<NA>
4,d,13.0,<NA>


In [44]:
ddf_a = dask_cudf.from_cudf(df_a, npartitions=2)
ddf_b = dask_cudf.from_cudf(df_b, npartitions=2)

merged = ddf_a.merge(ddf_b, on=['key'], how='left').compute()
merged

,key,vals_a,vals_b
0,a,10.0,100.0
1,c,12.0,101.0
2,b,11.0,<NA>
0,e,14.0,102.0
1,d,13.0,<NA>


## Append

Appending values from another `Series` or array-like object.

In [45]:
s.append(s)

/home/mmccarty/miniconda3/envs/cudf_dev/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2329: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(


0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64

In [46]:
ds2.append(ds2).compute()

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64

## Grouping

Like pandas, cuDF and Dask-cuDF support the Split-Apply-Combine groupby paradigm.

In [47]:
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]

ddf = dask_cudf.from_cudf(df, npartitions=2)

Grouping and then applying the `sum` function to the grouped data.

In [48]:
df.groupby('agg_col1').sum()

,a,b,c,agg_col2
agg_col1,,,,
1,90,100,90,4
0,100,90,100,3


In [49]:
ddf.groupby('agg_col1').sum().compute()

,a,b,c,agg_col2
agg_col1,,,,
1,90,100,90,4
0,100,90,100,3


Grouping hierarchically then applying the `sum` function to grouped data.

In [50]:
df.groupby(['agg_col1', 'agg_col2']).sum()

,,a,b,c
agg_col1,agg_col2,,,
1,0,54,60,54
0,0,73,60,73
1,1,36,40,36
0,1,27,30,27


In [51]:
ddf.groupby(['agg_col1', 'agg_col2']).sum().compute()

,,a,b,c
agg_col1,agg_col2,,,
1,1,36,40,36
0,0,73,60,73
1,0,54,60,54
0,1,27,30,27


Grouping and applying statistical functions to specific columns, using `agg`.

In [52]:
df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'})

,a,b,c
agg_col1,,,
1,18,10.0,90
0,19,9.0,100


In [53]:
ddf.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}).compute()

,a,b,c
agg_col1,,,
1,18,10.0,90
0,19,9.0,100


## Transpose

Transposing a dataframe, using either the `transpose` method or `T` property. Currently, all columns must have the same type. Transposing is not currently implemented in Dask-cuDF.

In [54]:
sample = cudf.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
sample

,a,b
0,1,4
1,2,5
2,3,6


In [55]:
sample.transpose()

,0,1,2
a,1,2,3
b,4,5,6


Time Series
------------

`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [56]:
import datetime as dt

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = cp.random.sample(len(date_df))

search_date = dt.datetime.strptime('2018-11-23', '%Y-%m-%d')
date_df.query('date <= @search_date')

,date,value
0,2018-11-20,0.986051
1,2018-11-21,0.232034
2,2018-11-22,0.397617
3,2018-11-23,0.103839


In [57]:
date_ddf = dask_cudf.from_cudf(date_df, npartitions=2)
date_ddf.query('date <= @search_date', local_dict={'search_date':search_date}).compute()

,date,value
0,2018-11-20,0.986051
1,2018-11-21,0.232034
2,2018-11-22,0.397617
3,2018-11-23,0.103839


Categoricals
------------

`DataFrames` support categorical columns.

In [58]:
gdf = cudf.DataFrame({"id": [1, 2, 3, 4, 5, 6], "grade":['a', 'b', 'b', 'a', 'a', 'e']})
gdf['grade'] = gdf['grade'].astype('category')
gdf

,id,grade
0,1,a
1,2,b
2,3,b
3,4,a
4,5,a
5,6,e


In [59]:
dgdf = dask_cudf.from_cudf(gdf, npartitions=2)
dgdf.compute()

,id,grade
0,1,a
1,2,b
2,3,b
3,4,a
4,5,a
5,6,e


Accessing the categories of a column. Note that this is currently not supported in Dask-cuDF.

In [60]:
gdf.grade.cat.categories

StringIndex(['a' 'b' 'e'], dtype='object')

Accessing the underlying code values of each categorical observation.

In [61]:
gdf.grade.cat.codes

0    0
1    1
2    1
3    0
4    0
5    2
dtype: uint8

In [62]:
dgdf.grade.cat.codes.compute()

0    0
1    1
2    1
3    0
4    0
5    2
dtype: uint8

Converting Data Representation
--------------------------------

## Pandas

Converting a cuDF and Dask-cuDF `DataFrame` to a pandas `DataFrame`.

In [63]:
df.head().to_pandas()

,a,b,c,agg_col1,agg_col2
0,0,19,0,1,1
1,1,18,1,0,0
2,2,17,2,1,0
3,3,16,3,0,1
4,4,15,4,1,0


In [64]:
ddf.compute().head().to_pandas()

,a,b,c,agg_col1,agg_col2
0,0,19,0,1,1
1,1,18,1,0,0
2,2,17,2,1,0
3,3,16,3,0,1
4,4,15,4,1,0


## Numpy

Converting a cuDF or Dask-cuDF `DataFrame` to a numpy `ndarray`.

In [65]:
df.to_numpy()

array([[ 0, 19,  0,  1,  1],
       [ 1, 18,  1,  0,  0],
       [ 2, 17,  2,  1,  0],
       [ 3, 16,  3,  0,  1],
       [ 4, 15,  4,  1,  0],
       [ 5, 14,  5,  0,  0],
       [ 6, 13,  6,  1,  1],
       [ 7, 12,  7,  0,  0],
       [ 8, 11,  8,  1,  0],
       [ 9, 10,  9,  0,  1],
       [10,  9, 10,  1,  0],
       [11,  8, 11,  0,  0],
       [12,  7, 12,  1,  1],
       [13,  6, 13,  0,  0],
       [14,  5, 14,  1,  0],
       [15,  4, 15,  0,  1],
       [16,  3, 16,  1,  0],
       [17,  2, 17,  0,  0],
       [18,  1, 18,  1,  1],
       [19,  0, 19,  0,  0]])

In [66]:
ddf.compute().to_numpy()

array([[ 0, 19,  0,  1,  1],
       [ 1, 18,  1,  0,  0],
       [ 2, 17,  2,  1,  0],
       [ 3, 16,  3,  0,  1],
       [ 4, 15,  4,  1,  0],
       [ 5, 14,  5,  0,  0],
       [ 6, 13,  6,  1,  1],
       [ 7, 12,  7,  0,  0],
       [ 8, 11,  8,  1,  0],
       [ 9, 10,  9,  0,  1],
       [10,  9, 10,  1,  0],
       [11,  8, 11,  0,  0],
       [12,  7, 12,  1,  1],
       [13,  6, 13,  0,  0],
       [14,  5, 14,  1,  0],
       [15,  4, 15,  0,  1],
       [16,  3, 16,  1,  0],
       [17,  2, 17,  0,  0],
       [18,  1, 18,  1,  1],
       [19,  0, 19,  0,  0]])

Converting a cuDF or Dask-cuDF `Series` to a numpy `ndarray`.

In [67]:
df['a'].to_numpy()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [68]:
ddf['a'].compute().to_numpy()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

## Arrow

Converting a cuDF or Dask-cuDF `DataFrame` to a PyArrow `Table`.

In [69]:
df.to_arrow()

pyarrow.Table
a: int64
b: int64
c: int64
agg_col1: int64
agg_col2: int64
----
a: [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]]
b: [[19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]]
c: [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]]
agg_col1: [[1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0]]
agg_col2: [[1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0]]

In [70]:
ddf.compute().to_arrow()

pyarrow.Table
a: int64
b: int64
c: int64
agg_col1: int64
agg_col2: int64
----
a: [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]]
b: [[19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]]
c: [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]]
agg_col1: [[1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0]]
agg_col2: [[1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0]]

Getting Data In/Out
------------------------

## CSV

Writing to a CSV file.

In [71]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_csv('example_output/foo.csv', index=False)

In [72]:
ddf.compute().to_csv('example_output/foo_dask.csv', index=False)

Reading from a csv file.

In [73]:
df = cudf.read_csv('example_output/foo.csv')
df

,a,b,c,agg_col1,agg_col2
0,0,19,0,1,1
1,1,18,1,0,0
2,2,17,2,1,0
3,3,16,3,0,1
4,4,15,4,1,0
5,5,14,5,0,0
6,6,13,6,1,1
7,7,12,7,0,0
8,8,11,8,1,0
9,9,10,9,0,1


In [74]:
ddf = dask_cudf.read_csv('example_output/foo_dask.csv')
ddf.compute()

,a,b,c,agg_col1,agg_col2
0,0,19,0,1,1
1,1,18,1,0,0
2,2,17,2,1,0
3,3,16,3,0,1
4,4,15,4,1,0
5,5,14,5,0,0
6,6,13,6,1,1
7,7,12,7,0,0
8,8,11,8,1,0
9,9,10,9,0,1


Reading all CSV files in a directory into a single `dask_cudf.DataFrame`, using the star wildcard.

In [75]:
ddf = dask_cudf.read_csv('example_output/*.csv')
ddf.compute()

,a,b,c,agg_col1,agg_col2
0,0,19,0,1,1
1,1,18,1,0,0
2,2,17,2,1,0
3,3,16,3,0,1
4,4,15,4,1,0
5,5,14,5,0,0
6,6,13,6,1,1
7,7,12,7,0,0
8,8,11,8,1,0
9,9,10,9,0,1


## Parquet

Writing to parquet files, using the CPU via PyArrow.

In [76]:
df.to_parquet('example_output/temp_parquet')

Reading parquet files with a GPU-accelerated parquet reader.

In [77]:
df = cudf.read_parquet('example_output/temp_parquet')
df

,a,b,c,agg_col1,agg_col2
0,0,19,0,1,1
1,1,18,1,0,0
2,2,17,2,1,0
3,3,16,3,0,1
4,4,15,4,1,0
5,5,14,5,0,0
6,6,13,6,1,1
7,7,12,7,0,0
8,8,11,8,1,0
9,9,10,9,0,1


Writing to parquet files from a `dask_cudf.DataFrame` using PyArrow under the hood.

In [78]:
ddf.to_parquet('example_files')  

(None,)

## ORC

Reading ORC files.

In [79]:
import os
from pathlib import Path
current_dir = os.path.dirname(os.path.realpath("__file__"))
cudf_root = Path(current_dir).parents[3]
file_path = os.path.join(cudf_root, "python", "cudf", "cudf", "tests", "data", "orc", "TestOrcFile.test1.orc")
file_path

'/home/ashwin/workspace/rapids/cudf/python/cudf/cudf/tests/data/orc/TestOrcFile.test1.orc'

In [80]:
df2 = cudf.read_orc(file_path)
df2

,boolean1,byte1,short1,int1,long1,float1,double1,bytes1,string1,middle,list,map
0,False,1,1024,65536,9223372036854775807,1.0,-15.0, ,hi,"{'list': [{'int1': 1, 'string1': 'bye'}, {'int...","[{'int1': 3, 'string1': 'good'}, {'int1': 4, '...",[]
1,True,100,2048,65536,9223372036854775807,2.0,-5.0,,bye,"{'list': [{'int1': 1, 'string1': 'bye'}, {'int...","[{'int1': 100000000, 'string1': 'cat'}, {'int1...","[{'key': 'chani', 'value': {'int1': 5, 'string..."


Dask Performance Tips
--------------------------------

Like Apache Spark, Dask operations are [lazy](https://en.wikipedia.org/wiki/Lazy_evaluation). Instead of being executed at that moment, most operations are added to a task graph and the actual evaluation is delayed until the result is needed.

Sometimes, though, we want to force the execution of operations. Calling `persist` on a Dask collection fully computes it (or actively computes it in the background), persisting the result into memory. When we're using distributed systems, we may want to wait until `persist` is finished before beginning any downstream operations. We can enforce this contract by using `wait`. Wrapping an operation with `wait` will ensure it doesn't begin executing until all necessary upstream operations have finished.

The snippets below provide basic examples, using `LocalCUDACluster` to create one dask-worker per GPU on the local machine. For more detailed information about `persist` and `wait`, please see the Dask documentation for [persist](https://docs.dask.org/en/latest/api.html#dask.persist) and [wait](https://docs.dask.org/en/latest/futures.html#distributed.wait). Wait relies on the concept of Futures, which is beyond the scope of this tutorial. For more information on Futures, see the Dask [Futures](https://docs.dask.org/en/latest/futures.html) documentation. For more information about multi-GPU clusters, please see the [dask-cuda](https://github.com/rapidsai/dask-cuda) library (documentation is in progress).

First, we set up a GPU cluster. With our `client` set up, Dask-cuDF computation will be distributed across the GPUs in the cluster.

In [81]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

2022-04-21 13:26:06,860 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 13:26:06,904 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 125.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33995,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 125.65 GiB
Comm: tcp://127.0.0.1:40479,Total threads: 1
Dashboard: http://127.0.0.1:38985/status,Memory: 62.82 GiB
Nanny: tcp://127.0.0.1:33447,


### Persisting Data
Next, we create our Dask-cuDF DataFrame and apply a transformation, storing the result as a new column.

In [82]:
nrows = 10000000

df2 = cudf.DataFrame({'a': cp.arange(nrows), 'b': cp.arange(nrows)})
ddf2 = dask_cudf.from_cudf(df2, npartitions=5)
ddf2['c'] = ddf2['a'] + 5
ddf2

,a,b,c
npartitions=5,,,
0,int64,int64,int64
2000000,...,...,...
...,...,...,...
8000000,...,...,...
9999999,...,...,...


In [83]:
!nvidia-smi

Thu Apr 21 13:26:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:15:00.0 Off |                  Off |
| 39%   52C    P2    51W / 250W |   1115MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GV100        Off  | 00000000:2D:00.0 Off |                  Off |
| 43%   

Because Dask is lazy, the computation has not yet occurred. We can see that there are twenty tasks in the task graph and we've used about 800 MB of memory. We can force computation by using `persist`. By forcing execution, the result is now explicitly in memory and our task graph only contains one task per partition (the baseline).

In [84]:
ddf2 = ddf2.persist()
ddf2

,a,b,c
npartitions=5,,,
0,int64,int64,int64
2000000,...,...,...
...,...,...,...
8000000,...,...,...
9999999,...,...,...


In [85]:
!nvidia-smi

Thu Apr 21 13:26:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:15:00.0 Off |                  Off |
| 39%   52C    P2    52W / 250W |   1115MiB / 32508MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GV100        Off  | 00000000:2D:00.0 Off |                  Off |
| 43%   

Because we forced computation, we now have a larger object in distributed GPU memory.

### Wait
Depending on our workflow or distributed computing setup, we may want to `wait` until all upstream tasks have finished before proceeding with a specific function. This section shows an example of this behavior, adapted from the Dask documentation.

First, we create a new Dask DataFrame and define a function that we'll map to every partition in the dataframe.

In [86]:
import random

nrows = 10000000

df1 = cudf.DataFrame({'a': cp.arange(nrows), 'b': cp.arange(nrows)})
ddf1 = dask_cudf.from_cudf(df1, npartitions=100)

def func(df):
    time.sleep(random.randint(1, 60))
    return (df + 5) * 3 - 11

This function will do a basic transformation of every column in the dataframe, but the time spent in the function will vary due to the `time.sleep` statement randomly adding 1-60 seconds of time. We'll run this on every partition of our dataframe using `map_partitions`, which adds the task to our task-graph, and store the result. We can then call `persist` to force execution.

In [87]:
results_ddf = ddf2.map_partitions(func)
results_ddf = results_ddf.persist()

However, some partitions will be done **much** sooner than others. If we had downstream processes that should wait for all partitions to be completed, we can enforce that behavior using `wait`.

In [88]:
wait(results_ddf)

DoneAndNotDoneFutures(done={<Future: finished, type: cudf.core.dataframe.DataFrame, key: ('func-c8623f55c898739bdfb89533682776dc', 0)>, <Future: finished, type: cudf.core.dataframe.DataFrame, key: ('func-c8623f55c898739bdfb89533682776dc', 3)>, <Future: finished, type: cudf.core.dataframe.DataFrame, key: ('func-c8623f55c898739bdfb89533682776dc', 1)>, <Future: finished, type: cudf.core.dataframe.DataFrame, key: ('func-c8623f55c898739bdfb89533682776dc', 2)>, <Future: finished, type: cudf.core.dataframe.DataFrame, key: ('func-c8623f55c898739bdfb89533682776dc', 4)>}, not_done=set())

With `wait`, we can safely proceed on in our workflow.